In [1]:
import numpy as np

def mar_to_state_space(Phi_list, Phi_0, r):
    """Convert MAR model to state-space form.
    
    Parameters:
    - Phi_list: List of MAR coefficient matrices [Phi_1, Phi_2, ..., Phi_k]
    - Phi_0: Constant term coefficient vector
    - r: Dimension of output vector
    
    Returns:
    - A: State transition matrix
    - B: Input (constant) matrix
    - C: Measurement matrix
    """
    k = len(Phi_list)  # Number of AR terms
    
    # Construct A matrix (Companion form)
    A = np.zeros((r * k, r * k))
    A[:r, :r * k] = np.hstack(Phi_list)  # First block row for AR coefficients
    for i in range(1, k):
        A[i * r:(i + 1) * r, (i - 1) * r:i * r] = np.eye(r)  # Identity blocks
    
    # Construct B matrix (constant input)
    B = np.zeros((r * k, 1))
    B[:r] = Phi_0.reshape(-1, 1)
    
    # Construct C matrix (outputs are first state)
    C = np.zeros((r, r * k))
    C[:, :r] = np.eye(r)
    
    return A, B, C

# Example usage
r = 2  # Dimension of output vector
t = 2  # Number of AR terms

# Example MAR coefficients (random for demonstration)
Phi_list = [np.array([[0.5, -0.1], [0.2, 0.3]]),
            np.array([[0.1, 0.05], [-0.1, 0.4]])]
Phi_0 = np.array([0.2, -0.1])

A, B, C = mar_to_state_space(Phi_list, Phi_0, r)

print("State Transition Matrix (A):\n", A)
print("\nInput Matrix (B):\n", B)
print("\nMeasurement Matrix (C):\n", C)


State Transition Matrix (A):
 [[ 0.5  -0.1   0.1   0.05]
 [ 0.2   0.3  -0.1   0.4 ]
 [ 1.    0.    0.    0.  ]
 [ 0.    1.    0.    0.  ]]

Input Matrix (B):
 [[ 0.2]
 [-0.1]
 [ 0. ]
 [ 0. ]]

Measurement Matrix (C):
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]]


In [3]:
import numpy as np
from scipy.stats import multivariate_normal

def compute_likelihood(y_t, y_pred, M_pred):
    """Compute the Gaussian likelihood P(y_t | I_t = j)."""
    return multivariate_normal.pdf(y_t, mean=y_pred, cov=M_pred)

def bayes_posterior(priors, likelihoods):
    """Compute posterior probabilities using Bayes' theorem."""
    numerator = priors * likelihoods
    denominator = np.sum(numerator)
    return numerator / denominator if denominator != 0 else np.zeros_like(numerator)

# Example Data
J = 3  # Number of modes
y_t = np.array([1.2, 0.5])  # Current measurement

y_preds = [
    np.array([1.0, 0.4]),  # Mode 1 predicted measurement
    np.array([1.1, 0.45]), # Mode 2 predicted measurement
    np.array([1.3, 0.6])   # Mode 3 predicted measurement
]

M_preds = [
    np.array([[0.1, 0.02], [0.02, 0.1]]),  # Mode 1 covariance
    np.array([[0.1, 0.01], [0.01, 0.1]]),  # Mode 2 covariance
    np.array([[0.2, 0.05], [0.05, 0.2]])   # Mode 3 covariance
]

priors = np.array([0.33, 0.33, 0.34])  # Uniform priors for all modes

# Compute likelihoods for each mode
likelihoods = np.array([compute_likelihood(y_t, y_preds[j], M_preds[j]) for j in range(J)])

# Compute posterior probabilities
posteriors = bayes_posterior(priors, likelihoods)

# Find the most probable mode
best_mode = np.argmax(posteriors) + 1

print("Likelihoods:", likelihoods)
print("Posteriors:", posteriors)
print("Best Mode:", best_mode)


Likelihoods: [1.30521663 1.50930969 0.78964651]
Posteriors: [0.35975196 0.41600536 0.22424268]
Best Mode: 2


In [15]:
import numpy as np
from scipy.stats import norm

# Define system modes (example: two modes with different system matrices)
modes = {
    "Mode 1": np.array([[1.5, 0.5], [0.3, 1.1]]), 
    "Mode 2": np.array([[0.8, 0.2], [0.1, 0.9]])
}

# Prior probabilities (assume equal probability initially)
prior_probs = {"Mode 1": 0.5, "Mode 2": 0.5}

# Generate sample observations (assume some input sequence)
u_t = np.array([[1], [2]])  # Example input
y_t_observed = np.array([[1.8], [2.5]])  # Example output

sigma = 0.1  # Noise variance

# Compute likelihoods for each mode
likelihoods = {}
for mode, B in modes.items():
    y_pred = B @ u_t  # Predicted output
    likelihood = norm.pdf(np.linalg.norm(y_t_observed - y_pred), scale=sigma)
    likelihoods[mode] = likelihood

# Compute posterior probabilities
posteriors = {mode: (likelihoods[mode] * prior_probs[mode]) for mode in modes}

# Normalize posteriors
total_prob = sum(posteriors.values())
posteriors = {mode: post / total_prob for mode, post in posteriors.items()}

# Identify most likely mode
detected_mode = max(posteriors, key=posteriors.get)

# Print results
print("Mode probabilities:", posteriors)
print("Detected Mode:", detected_mode)


Mode probabilities: {'Mode 1': 0.9999898700090192, 'Mode 2': 1.0129990980874065e-05}
Detected Mode: Mode 1
